In [2]:
import pandas as pd

df_18 = pd.read_csv('../MIGTABO19X.csv',sep="|")
df_columns_18 = pd.read_csv('Campos MIGTAB019.csv',sep=";")
#df_18.drop('Unnamed: 3',axis=1,inplace= True)
df_18.columns = df_columns_18['NombreColumna'].tolist()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

C:\Users\Fabian - PC\AppData\Local\Temp\ipykernel_19196\3853580854.py:3: DtypeWarning: Columns (2,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_18 = pd.read_csv('../MIGTABO19X.csv',sep="|")


In [3]:
df_18.head(5)

,ro_operacion,ro_concepto,ro_signo,ro_factor,ro_referencial,ro_signo_reajuste,ro_factor_reajuste,ro_referencial_reajuste,ro_valor,ro_porcentaje_efa,ro_garantia,ro_financiado
0,5001,INT,NaN,0.0,T_FIJA,NaN,NaN,NaN,0.0,29.3334,0,NaN
1,5001,IMO,NaN,0.0,T_USURA,NaN,0.0,NaN,0.0,0.0000,0,NaN
2,5002,CAP,NaN,0.0,NaN,NaN,0.0,NaN,3672326.0,0.0000,0,NaN
3,5002,INT,NaN,0.0,T_FIJA,NaN,NaN,NaN,0.0,24.9962,0,NaN
4,5002,IMO,NaN,0.0,T_USURA,NaN,0.0,NaN,0.0,0.0000,0,NaN


#### Validación Campos obligatorios que no vengan nulos

In [ ]:
df_columnasObligatorias = df_columns_18[df_columns_18['Obligatorio'] == 'S']['NombreColumna'].tolist()
df_Final = df_18[df_columnasObligatorias]
#df_Final.shape
nulos = df_Final.isnull().sum().reset_index()
nulos.columns = ['NombreColumna','CantidadNulos']
df_Columnas = nulos[nulos['CantidadNulos']> 0 ]['NombreColumna'].head(4).tolist()


[]

In [14]:
errores = {}

In [20]:
df_18["ro_operacion"] = pd.to_numeric(df_18["ro_operacion"], errors="coerce")
errores["ro_operacion"] = df_18[df_18["ro_operacion"].isnull() | (df_18["ro_operacion"] == 0)]


In [22]:
errores["ro_concepto"] = df_18[df_18["ro_concepto"].isnull() | (df_18["ro_concepto"].astype(str).str.strip() == "")]


In [ ]:
# Validación 3: ro_signo
df_18[
    ~df_18["ro_signo"].isin(["+", "-"]) & 
    df_18["ro_signo"].notna() &           
    (df_18["ro_signo"].astype(str).str.strip() != "") 
]['ro_operacion']

Series([], Name: ro_operacion, dtype: int64)

Factor Rubro

In [11]:
df_18[(df_18["ro_concepto"] == "CAPITAL") & (df_18["ro_factor"] != 0)]['ro_operacion']

Series([], Name: ro_operacion, dtype: int64)

Tasa Referencial

In [ ]:
df_18[
    (
        (df_18["ro_concepto"] == "IMO") & 
        (df_18["ro_referencial"] != "T_USURA")
    ) | 
    (
        (df_18["ro_concepto"] == "CAP") &
        df_18["ro_referencial"].notna()
    )
]['ro_operacion']

,ro_operacion,ro_concepto,ro_signo,ro_factor,ro_referencial,ro_signo_reajuste,ro_factor_reajuste,ro_referencial_reajuste,ro_valor,ro_porcentaje_efa,ro_garantia,ro_financiado


ro_signo_reajuste

In [25]:
df_18[
    ~df_18["ro_signo"].isin(["+", "-","/","*"]) & 
    df_18["ro_signo"].notna() &           
    (df_18["ro_signo"].astype(str).str.strip() != "") 
]['ro_operacion']

Series([], Name: ro_operacion, dtype: int64)

ro_factor_reajuste

In [28]:
df_18[df_18["ro_concepto"].isin(["CAP","IMO"]) & df_18["ro_signo_reajuste"] != 0]['ro_operacion']

Series([], Name: ro_operacion, dtype: int64)

ro_referencial_reajuste

In [30]:
df_18[df_18["ro_concepto"].isin(["CAP","IMO"]) & df_18["ro_signo_reajuste"].notna()]['ro_operacion']

Series([], Name: ro_operacion, dtype: int64)

ro_valor

In [ ]:
df_18[
    (
        df_18["ro_concepto"].isin(["INT", "IMO"]) & 
        (df_18["ro_valor"] != 0)
    ) |
    (
        (df_18["ro_concepto"] == "CAP") & 
        (df_18["ro_valor"] < 0)
    )
]["ro_operacion"]


Series([], Name: ro_operacion, dtype: int64)

ro_garantia

In [40]:
df_18[df_18["ro_garantia"] != 0]["ro_operacion"]

Series([], Name: ro_operacion, dtype: int64)

ro_financiado

In [41]:
df_18[df_18["ro_financiado"].notna()]["ro_operacion"]

Series([], Name: ro_operacion, dtype: int64)